# 1. Backfill

## 1.1. Environment Setup
Handle repository cloning, dependency installation, and set up Python path.

In [ ]:
import sys
from pathlib import Path
import hopsworks
import warnings

warnings.filterwarnings("ignore", module="IPython")

def clone_repository() -> None:
    repo_dir = Path("pm25-forecast-openmeteo-aqicn")
    if repo_dir.exists():
        print(f"Repository already exists at {repo_dir.absolute()}")
        %cd pm25-forecast-openmeteo-aqicn
    else:
        print("Cloning repository...")
        !git clone https://github.com/KristinaPalmquist/pm25-forecast-openmeteo-aqicn.git
        %cd pm25-forecast-openmeteo-aqicn

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml


root_dir = Path().absolute()
for folder in ("src", "airquality", "notebooks"):
    if root_dir.parts[-1:] == (folder,):
        root_dir = Path(*root_dir.parts[:-1])
root_dir = str(root_dir)

if root_dir not in sys.path:
    sys.path.append(root_dir)

from utils import config

settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")
HOPSWORKS_API_KEY = settings.HOPSWORKS_API_KEY.get_secret_value()
project = hopsworks.login(engine="python", api_key_value=HOPSWORKS_API_KEY)
fs = project.get_feature_store()

## 1.2. Imports

In [ ]:
import datetime
import importlib
import time
import requests
import os
import pandas as pd
import great_expectations as ge
import hopsworks
from utils import airquality
importlib.reload(airquality)
import json
import warnings
warnings.filterwarnings("ignore")

## 1.3. Setup
Configure Hopsworks connection, feature store access, and AQICN API key handling

In [ ]:
today = datetime.date.today()

if settings.AQICN_API_KEY is None:
    print("You need to set AQICN_API_KEY either in this cell or in ~/.env")
    sys.exit(1)

AQICN_API_KEY = settings.AQICN_API_KEY.get_secret_value()

secrets = hopsworks.get_secrets_api()
try:
    secret = secrets.get_secret("AQICN_API_KEY")
    if secret is not None:
        secret.delete()
        print("Replacing existing AQICN_API_KEY")
except hopsworks.RestAPIError as e:
    if hasattr(e, "error_code") and getattr(e, "error_code", None) == 160048:
        pass
    elif "Could not find Secret" in str(e):
        pass
    else:
        raise

secrets.create_secret("AQICN_API_KEY", AQICN_API_KEY)

## 1.4. Processing Mode
This notebook processes all sensors that have CSV files in the `data` folder.

## 1.5. Data Validation Setup
Creates Great Expectations validation suites for air quality and weather data with column value constraints.

In [ ]:
aq_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="aq_expectation_suite"
)
aq_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column": "pm25",
            "min_value": -0.1,
            "max_value": 500.0,
            "strict_min": True,
        },
    )
)

weather_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="weather_expectation_suite"
)
def expect_greater_than_zero(col):
    weather_expectation_suite.add_expectation(
        ge.core.ExpectationConfiguration(
            expectation_type="expect_column_min_to_be_between",
            kwargs={
                "column": col,
                "min_value": -0.1,
                "max_value": 1000.0,
                "strict_min": True,
            },
        )
    )

expect_greater_than_zero("precipitation_sum")
expect_greater_than_zero("wind_speed_10m_max")

## 1.6. Helper Methods
Data processing functions - clean air quality data and fetch historical weather data with API rate limiting and retry logic.

In [ ]:
def clean_and_append_data(df, street, city, country, feed_url, sensor_id):
    """
    Remove any unused columns, set the daily median value to pm25. Remove NaN's and append the metadata.
    """
    clean_df = pd.DataFrame()
    clean_df["date"] = pd.to_datetime(df["date"])
    clean_df["pm25"] = df["median"]
    clean_df = clean_df.dropna(subset=["pm25"])
    clean_df["sensor_id"] = sensor_id
    clean_df["street"] = street
    clean_df["city"] = city
    clean_df["country"] = country
    clean_df["feed_url"] = feed_url
    return clean_df

def get_historical_weather(city, df, today, feed_url, sensor_id):
    earliest_aq_date = pd.Series.min(df["date"])
    earliest_aq_date = earliest_aq_date.strftime("%Y-%m-%d")
    
    response = requests.get(f"{feed_url}/?token={AQICN_API_KEY}")
    data = response.json()

    # Handle AQICN API error status or missing city
    if ("status" in data.get("data", {}) and data["data"]["status"] == "error") or "city" not in data.get("data", {}):
        print(f"Skipping sensor {sensor_id}: AQICN API error or unknown ID. Response: {data}")
        return None, None, None 
    try:
        latitude, longitude = airquality.get_sensor_coordinates(feed_url, sensor_id, AQICN_API_KEY)
        
    except Exception as e:
        raise ValueError(f"Failed to get coordinates for sensor {sensor_id}: {e}")
    
    max_retries = 5
    attempt = 0
    while attempt < max_retries:
        try:
            weather_df = airquality.get_historical_weather(
                city, earliest_aq_date, str(today), latitude, longitude
            )
            weather_df["sensor_id"] = sensor_id
            weather_df["city"] = city
            weather_df["latitude"] = latitude
            weather_df["longitude"] = longitude
            return weather_df, latitude, longitude
        except Exception as e:
            if hasattr(e, "args") and any(
                "Minutely API request limit exceeded" in str(a) for a in e.args
            ):
                wait_time = 70
                print(
                    f"OpenMeteo API limit exceeded, retrying in {wait_time} seconds... (Attempt {attempt + 1} of {max_retries})"
                )
                time.sleep(wait_time)
                attempt += 1
            elif "Minutely API request limit exceeded" in str(e):
                wait_time = 70
                print(
                    f"OpenMeteo API limit exceeded, retrying in {wait_time} seconds... (Attempt {attempt + 1} of {max_retries})"
                )
                time.sleep(wait_time)
                attempt += 1
            else:
                raise
    raise RuntimeError(
        "Failed to obtain historical weather after multiple retries due to API rate limits."
    )

## 1.7. Hopsworks
Feature Group Management - functions to create and manage air quality and weather feature groups in Hopsworks, including schema descriptions.

In [ ]:
def create_air_quality_feature_group():
    air_quality_fg = fs.get_or_create_feature_group(
        name="air_quality_all",
        description="Air Quality characteristics of each day for all sensors",
        version=1,
        primary_key=["sensor_id"],
        event_time="date",
        expectation_suite=aq_expectation_suite,
    )
    return air_quality_fg

In [ ]:
# def remove_duplicates(new_df=None):
#     """
#     Remove duplicate entries from the Hopsworks feature group, keeping only the latest for each (sensor_id, date).
#     If new_df is provided, also remove any rows from new_df that would duplicate (sensor_id, date) already present in the feature group.
#     Returns a deduplicated DataFrame for upload (if new_df is provided), else None.
#     """
#     # Try to read from the feature group, handle empty case
#     try:
#         df = air_quality_fg.read()
#         feature_group_empty = df.empty
#     except Exception as e:
#         print("Feature group is empty or cannot be read. Skipping FG deduplication.")
#         feature_group_empty = True
#         df = None

#     # Deduplicate new_df in-memory before upload
#     if new_df is not None:
#         # Remove duplicates within new_df itself (by sensor_id and date)
#         new_df = new_df.sort_values(['sensor_id', 'date'])
#         new_df = new_df.drop_duplicates(subset=['sensor_id', 'date'], keep='last')
#         if not feature_group_empty:
#             # Remove rows from new_df that already exist in the feature group
#             df_keys = df[['sensor_id', 'date']].drop_duplicates()
#             new_df = new_df.copy()
#             df_keys = df_keys.copy()
#             # Ensure both are timezone-naive
#             new_df['date'] = pd.to_datetime(new_df['date']).dt.tz_localize(None)
#             df_keys['date'] = pd.to_datetime(df_keys['date']).dt.tz_localize(None)
#             merged = new_df.merge(df_keys, on=['sensor_id', 'date'], how='left', indicator=True)
#             deduped_new_df = merged[merged['_merge'] == 'left_only'].drop(columns=['_merge'])
#             print(f"Filtered out {len(new_df) - len(deduped_new_df)} rows from new data that already exist in feature group.")
#             new_df = deduped_new_df
#         # Final check: ensure no duplicates remain in new_df
#         final_dupes = new_df.duplicated(subset=['sensor_id', 'date'], keep=False)
#         if final_dupes.any():
#             print("Warning: Duplicates still present in new_df after deduplication. Removing all but the last occurrence.")
#             new_df = new_df.drop_duplicates(subset=['sensor_id', 'date'], keep='last')
#         return new_df

#     # If feature group is not empty, deduplicate it
#     if not feature_group_empty:
#         df_sorted = df.sort_values(['sensor_id', 'date'])
#         mask = df_sorted.duplicated(subset=['sensor_id', 'date'], keep='last')
#         duplicates = df_sorted[mask]
#         print(f"Found {len(duplicates)} duplicate rows to delete in feature group.")
#         for _, row in duplicates.iterrows():
#             air_quality_fg.delete_record({"sensor_id": row["sensor_id"], "date": row["date"]})
#         print("Duplicate rows deleted from feature group.")
#     return None

In [ ]:
def update_air_quality_description(air_quality_fg):
    air_quality_fg.update_feature_description(
        "date", "Date of measurement of air quality"
    )
    air_quality_fg.update_feature_description(
        "sensor_id", "AQICN sensor identifier (e.g., 59893)"
    )
    air_quality_fg.update_feature_description(
        "country",
        "Country where the air quality was measured (sometimes a city in aqicn.org)",
    )
    air_quality_fg.update_feature_description(
        "city", "City where the air quality was measured"
    )
    air_quality_fg.update_feature_description(
        "street", "Street in the city where the air quality was measured"
    )
    air_quality_fg.update_feature_description(
        "pm25",
        "Particles less than 2.5 micrometers in diameter (fine particles) pose health risk",
    )
    air_quality_fg.update_feature_description(
        "pm25_rolling_3d",
        "3-day rolling mean of PM2.5 from previous days (lagged by 1 day for point-in-time correctness).",
    )
    air_quality_fg.update_feature_description(
        "pm25_lag_1d",
        "PM2.5 value from 1 day ago.",
    )
    air_quality_fg.update_feature_description(
        "pm25_lag_2d",
        "PM2.5 value from 2 days ago.",
    )
    air_quality_fg.update_feature_description(
        "pm25_lag_3d",
        "PM2.5 value from 3 days ago.",
    )
    air_quality_fg.update_feature_description(
        "pm25_nearby_avg",
        "Average PM2.5 value from the 3 closest sensors.",
    )

In [ ]:
def create_and_insert_air_quality_data(df):
    air_quality_fg = create_air_quality_feature_group()
    air_quality_fg.insert(df)
    update_air_quality_description(air_quality_fg)
    # # remove_duplicates()
    # deduped_df = remove_duplicates(df)
    # # Check for dict-typed columns
    # for col in deduped_df.columns:
    #     if deduped_df[col].apply(lambda x: isinstance(x, dict)).any():
    #         print(f"Warning: Column '{col}' contains dict values!")
    # # air_quality_fg.insert(deduped_df)
    # update_air_quality_description(air_quality_fg)

In [ ]:
def create_weather_feature_group():
    weather_fg = fs.get_or_create_feature_group(
        name="weather_all",
        description="Weather characteristics of each day for all sensors",
        version=1,
        primary_key=["sensor_id"],
        event_time="date",
        expectation_suite=weather_expectation_suite,
    )
    return weather_fg

In [ ]:
def update_weather_description(weather_fg):
    weather_fg.update_feature_description("date", "Date of measurement of weather")
    weather_fg.update_feature_description(
        "sensor_id", "AQICN sensor identifier (e.g., 59893)"
    )
    weather_fg.update_feature_description(
        "city", "City where weather is measured/forecast for"
    )
    weather_fg.update_feature_description(
        "temperature_2m_mean", "Temperature in Celsius"
    )
    weather_fg.update_feature_description(
        "precipitation_sum", "Precipitation (rain/snow) in mm"
    )
    weather_fg.update_feature_description(
        "wind_speed_10m_max", "Wind speed at 10m abouve ground"
    )
    weather_fg.update_feature_description(
        "wind_direction_10m_dominant", "Dominant Wind direction over the dayd"
    )
    weather_fg.update_feature_description(
        "latitude", "Latitude of sensor location used for weather retrieval"
    )
    weather_fg.update_feature_description(
        "longitude", "Longitude of sensor location used for weather retrieval"
    )

In [ ]:
def create_and_insert_weather_data(df):
    weather_fg = create_weather_feature_group()
    weather_fg.insert(df)
    update_weather_description(weather_fg)

## 1.8. Script
Main processing logic - processes all sensors in the data folder, cleans data, fetches weather data, adds rolling averages and lagged features, and combines all data

In [ ]:
all_aq_dfs = []
all_weather_dfs = []
locations = {}

# Process all files in data directory
data_dir = os.path.join(root_dir, "data")
dir_list = os.listdir(data_dir)
for file in dir_list:
    if file.endswith(".csv"):
        file_path = os.path.join(data_dir, file)
        aq_df_raw, street, city, country, feed_url, sensor_id = airquality.read_sensor_data(file_path, AQICN_API_KEY)
        aq_df = clean_and_append_data(aq_df_raw, street, city, country, feed_url, sensor_id)
        weather_df, latitude, longitude = get_historical_weather(
            city, aq_df, today, feed_url, sensor_id
        )
        all_aq_dfs.append(aq_df)
        all_weather_dfs.append(weather_df)
        locations[sensor_id] = {
            "country": country,
            "city": city,
            "street": street,
            "aqicn_url": feed_url,
            "latitude": latitude,
            "longitude": longitude,
        }

# Concatenate into single, uniform dfs
aq_df_all = pd.concat(all_aq_dfs, ignore_index=True)
weather_df_all = pd.concat(all_weather_dfs, ignore_index=True)
aq_df_all = airquality.add_rolling_window_feature(aq_df_all, window_days=3, column="pm25", new_column="pm25_rolling_3d")
aq_df_all = airquality.add_lagged_features(aq_df_all, column="pm25", lags=[1, 2, 3])
aq_df_all = airquality.add_nearby_sensor_feature(aq_df_all, locations, column="pm25_lag_1d", n_closest=3)

## 1.9. Exploration

In [ ]:
print("🔍 AIR QUALITY DATA EXPLORATION")
print("="*40)
print(f"Shape: {aq_df_all.shape}")
print(f"Date range: {aq_df_all['date'].min().date()} to {aq_df_all['date'].max().date()}")
print(f"Number of unique sensors: {aq_df_all['sensor_id'].nunique()}")
print(f"Countries: {aq_df_all['country'].unique()}")
print(f"Cities: {aq_df_all['city'].nunique()} unique cities")

print("\n📊 PM2.5 Statistics:")
print(aq_df_all['pm25'].describe())
print(f"Missing values: {aq_df_all['pm25'].isna().sum()}")

print("\n📈 Engineered Features Statistics:")
for col in ['pm25_rolling_3d', 'pm25_lag_1d', 'pm25_lag_2d', 'pm25_lag_3d', 'pm25_nearby_avg']:
    if col in aq_df_all.columns:
        missing = aq_df_all[col].isna().sum()
        print(f"{col}: {missing} missing values ({missing/len(aq_df_all)*100:.1f}%)")

In [ ]:
print("🌤️ WEATHER DATA EXPLORATION") 
print("="*40)
print(f"Shape: {weather_df_all.shape}")
print(f"Date range: {weather_df_all['date'].min().date()} to {weather_df_all['date'].max().date()}")
print(f"Number of unique sensors: {weather_df_all['sensor_id'].nunique()}")

print("\n🌡️ Weather Statistics:")
for col in ['temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max', 'wind_direction_10m_dominant']:
    if col in weather_df_all.columns:
        print(f"{col}:")
        print(f"  Range: {weather_df_all[col].min():.2f} to {weather_df_all[col].max():.2f}, Mean: {weather_df_all[col].mean():.2f}, Missing: {weather_df_all[col].isna().sum()}")

print("\n📍 Geographic Coverage:")
print(f"Latitude range: {weather_df_all['latitude'].min():.3f} to {weather_df_all['latitude'].max():.3f}, Longitude range: {weather_df_all['longitude'].min():.3f} to {weather_df_all['longitude'].max():.3f}")

In [ ]:
print("🔗 DATA QUALITY & RELATIONSHIPS")
print("="*40)

# Overall data completeness
sensor_day_counts = aq_df_all.groupby('sensor_id')['date'].count()
total_records = len(aq_df_all)
data_completeness = (1 - aq_df_all['pm25'].isna().sum() / total_records) * 100

print(f"📊 Overall Data Quality:")
print(f"Total records: {total_records:,}")
print(f"Data completeness: {data_completeness:.1f}%")
print(f"Days per sensor - Min: {sensor_day_counts.min()}, Median: {sensor_day_counts.median():.0f}, Max: {sensor_day_counts.max()}")
print(f"Sensors with <30 days: {(sensor_day_counts < 30).sum()}, >365 days: {(sensor_day_counts > 365).sum()}")

# Extreme values summary
extreme_count = (aq_df_all['pm25'] > 100).sum()
very_high_count = (aq_df_all['pm25'] > 50).sum()
print(f"\n⚠️ Air Quality Levels:")
print(f"Extreme readings (>100 μg/m³): {extreme_count} ({extreme_count/total_records*100:.1f}%)")
print(f"Very high readings (>50 μg/m³): {very_high_count} ({very_high_count/total_records*100:.1f}%)")

# Seasonal patterns
if len(aq_df_all) > 0:
    # Create temporary month column without modifying original DataFrame
    temp_months = pd.to_datetime(aq_df_all['date']).dt.month
    monthly_pm25 = aq_df_all.groupby(temp_months)['pm25'].mean()
    print(f"\n🗓️ Seasonal Patterns (PM2.5 μg/m³):")
    seasons = {(12,1,2): "Winter", (3,4,5): "Spring", (6,7,8): "Summer", (9,10,11): "Autumn"}
    for months, season in seasons.items():
        season_avg = monthly_pm25[monthly_pm25.index.isin(months)].mean()
        print(f"  {season}: {season_avg:.1f}")

## 1.10. Store Sensor Location
Create Hopsworks secrets for each sensor's location metadata (coordinates, address, etc.)

In [ ]:
for sensor_id, location in locations.items():
    secret_name = f"SENSOR_LOCATION_JSON_{sensor_id}"
    location_str = json.dumps(location)
    
    try:
        secret = secrets.get_secret(secret_name)
        if secret is not None:
            secret.delete()
    except hopsworks.RestAPIError as e:
        if hasattr(e, "error_code") and getattr(e, "error_code", None) == 160048:
            pass
        elif "Could not find Secret" in str(e):
            pass
        else:
            raise
    
    secrets.create_secret(secret_name, location_str)

## 1.11. Upload to Hopsworks
Insert the processed air quality and weather data into Hopsworks feature groups

In [ ]:
create_and_insert_air_quality_data(aq_df_all)
create_and_insert_weather_data(weather_df_all)